In [166]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain.agents.middleware import wrap_model_call, ModelRequest, ToolCallLimitMiddleware
from langchain.messages import SystemMessage

from langchain_tavily import TavilySearch
from typing import Callable, Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from pprint import pprint
import os

In [167]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [168]:
wikitool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(
        doc_content_chars_max=2000,
        top_k_results=5
    )
)

tavilytool = TavilySearch(
    maxResults = 5,
    topic = "general"
)

tools = [tavilytool, wikitool]

In [169]:
llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai"
)

In [170]:
class ToolDecision(BaseModel):
    """  
        Class will be used by LLM in toolcall function. toolSelection Field is a Literal eith contains values "WIKI" OR "TAVILY", based on LLM output.
    """

    toolSelection: Literal["WIKI","TAVILY"] = Field(description="toolSelection will store only two constant WIKI or TAVILY")

In [171]:
settoollimit = ToolCallLimitMiddleware(
    run_limit=3, exit_behavior="end"
)

In [ ]:
@wrap_model_call
def toolcall(request: ModelRequest, handler: Callable) -> Callable:
    """  
        Function is used to select the tool based on request. 
        If User request is related to Power BI or LLM then it uses Wikitool else it choose tavily Tool.
    """

    msg = request.messages[-1].content

    systemprompt = f"""  
                        You are a routing assistant. 
                        Based on the user query, your ONLY task is to output exactly one word: either "WIKI" or "TAVILY".

                            Decision Logic:
                            - Output "WIKI" if the query is related to Power BI, Business Intelligence, or Large Language Models (LLMs).
                            - Output "TAVILY" for all other topics or general web searches.

                            Constraint:
                            - Do NOT provide explanations, pleasantries, or punctuation. 
                            - Output ONLY the uppercase word.

                            Query: {msg}
                            Output:

                            ### EXAMPLES:
                            Query: "How do I create a DAX measure in Power BI?"
                            Output: WIKI

                            ### EXAMPLES:
                            Query: "What is Sun"
                            Output: TAVILY
                    """

    llmjudge = llm.with_structured_output(ToolDecision)
    decision = llmjudge.invoke([SystemMessage(content=systemprompt)])
    print(msg,'\n',decision)

    if decision.toolSelection == "WIKI":
        new_request = request.override(tools= [wikitool], tool_choice = "required")
    else:
        new_request = request.override(tools=[tavilytool], tool_choice="required")

    return handler(new_request)
    

In [173]:
llmagent = create_agent(
    model=llm,
    middleware=[toolcall, settoollimit],
    tools=tools
)



In [174]:
result = llmagent.invoke({"messages":"Tell Me about India"})
for item in result['messages']:
    item.pretty_print()

Tell Me about India 
 toolSelection='TAVILY'
{"query": "India", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://en.wikipedia.org/wiki/India", "title": "India - Wikipedia", "content": "India, officially the Republic of India, is a country in South Asia. It is the seventh-largest country by area; the most populous country \"List of countries by population (United Nations)\") since 2023; and, since its independence in 1947, the world's most populous democracy. Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west; China, Nepal, and Bhutan to the north; and Bangladesh and [...] India is a megadiverse country, a term employed for 17 countries that display high biological diversity and contain many species exclusively indigenous \"Indigenous (ecology)\"), or endemic, to them. India is the habitat for 8.6% of all mammals, 13.7% of bird species,